# IBM Data Science Capstone Project Final Notebook


In this notebook, I will be creating clusters to find the most suitable location to open a bar in Toronto, Canada.

In [1]:
#importing necessary libraries
import requests
import pandas as pd

In [2]:
#scrapping neighborhoods in Canada
url  = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
page = requests.get(url)
if page.status_code == 200:
    print('Page download successful')
else:
    print('Page download error. Error code: {}'.format(page.status_code))

Page download successful


In [3]:
df_html = pd.read_html(url, header=0, na_values = ['Not assigned'])[0]
df_html.head()

,Postal code,Borough,Neighborhood
0,M1A,NaN,NaN
1,M2A,NaN,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


In [4]:
#Drop the the rows on which the Borough is empty
df_html.dropna(subset=['Borough'], inplace=True)

In [5]:
#Check Neighborhood is empty but Borough exists
n_empty_neighborhood = df_html[df_html['Neighborhood'].isna()].shape[0]
print('Number of rows on which Neighborhood column is empty: {}'.format(n_empty_neighborhood))

Number of rows on which Neighborhood column is empty: 0


In [6]:
#Group by Postcode / Borough
df_postcodes = df_html.groupby(['Postal code','Borough']).Neighborhood.agg([('Neighborhood', ', '.join)])
df_postcodes.reset_index(inplace=True)
df_postcodes.head(5)

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,Malvern / Rouge
1,M1C,Scarborough,Rouge Hill / Port Union / Highland Creek
2,M1E,Scarborough,Guildwood / Morningside / West Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [7]:
#fixing the format of Neighborhood column
df_postcodes["Neighborhood"] = df_postcodes['Neighborhood'].apply(lambda x : x.replace('/', ','))
df_postcodes.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
print('The shape of the dataset is:',df_postcodes.shape)

The shape of the dataset is: (103, 3)


In [9]:
#to make it accessible, we will store this in csv format.
#Export to .CSV
df_postcodes.to_csv('Toronto_Postcodes.csv')

In [10]:
import numpy as np

In [11]:
#Read CSV file which contains the data of Latitude and Longitude 
url_csv = 'http://cocl.us/Geospatial_data'
df_coordinates = pd.read_csv(url_csv)
df_coordinates.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
#use the previously cleaned data
df_neighborhoods = pd.read_csv('Toronto_Postcodes.csv',index_col=[0])
df_neighborhoods.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:
# Make sure both dataframes have the same column names
df_coordinates.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)
df_neighborhoods.rename(columns={'Postal code': 'PostalCode'}, inplace=True)

In [14]:
# Merge both datasets
df_neighborhoods_coordinates = pd.merge(df_neighborhoods, df_coordinates, on='PostalCode')
df_neighborhoods_coordinates.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [15]:
# Check coordinates for a couple of neighborhoods
df_neighborhoods_coordinates[(df_neighborhoods_coordinates['PostalCode']=='M5G') |
                             (df_neighborhoods_coordinates['PostalCode']=='M2H') ]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
17,M2H,North York,Hillcrest Village,43.803762,-79.363452
57,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383


In [16]:
#Export to .CSV
df_neighborhoods_coordinates.to_csv('Toronto_Postcodes_2.csv')


In [17]:
from sklearn.cluster import KMeans

In [18]:
# Read .csv file from above
df = pd.read_csv('Toronto_Postcodes_2.csv', index_col=0)
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [19]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.


In [20]:
#count Bourough and Neighborhood
df.groupby('Borough').count()['Neighborhood']

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
East York            5
Etobicoke           12
Mississauga          1
North York          24
Scarborough         17
West Toronto         6
York                 5
Name: Neighborhood, dtype: int64

In [21]:
df_toronto = df[df['Borough'].str.contains('Toronto')]
df_toronto.reset_index(inplace=True)
df_toronto.drop('index', axis=1, inplace=True)
df_toronto.head()


C:\Users\aishi\Anaconda3\lib\site-packages\pandas\core\frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West , Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar , The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


In [22]:
#Check the number of neighborhoods
print(df_toronto.groupby('Borough').count()['Neighborhood'])

Borough
Central Toronto      9
Downtown Toronto    19
East Toronto         5
West Toronto         6
Name: Neighborhood, dtype: int64


In [23]:
#Create list with the Boroughs (to be used later)
boroughs = df_toronto['Borough'].unique().tolist()

In [24]:
#Obtain the coordinates from the dataset itself, just averaging Latitude/Longitude of the current dataset 
lat_toronto = df_toronto['Latitude'].mean()
lon_toronto = df_toronto['Longitude'].mean()
print('The geographical coordinates of Toronto are {}, {}'.format(lat_toronto, lon_toronto))

The geographical coordinates of Toronto are 43.66713498717948, -79.38987324871795


In [25]:
borough_color = {}
for borough in boroughs:
    borough_color[borough]= '#%02X%02X%02X' % tuple(np.random.choice(range(256), size=3)) #Random color

In [26]:
import folium # map rendering library

In [27]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto['Latitude'], df_toronto['Longitude'], df_toronto['Borough'], df_toronto['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=borough_color[borough],
        fill_color=borough_color[borough],
        fill_opacity=0.7).add_to(map_toronto) 
    
map_toronto

# Getting Venues Data using Foursquare

In [28]:
CLIENT_ID = 'GXB3HZYAZFDTWD540G3OLDGPM54YHJKEDGHS5NM5W2Y2HMO4' # your Foursquare ID
CLIENT_SECRET = 'ZXQAVEYIWIHJNDMKEVGCG0XJ2ZGWO52LUYGJU2DWPQAM5X4N' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

In [31]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    
    return(nearby_venues)

In [33]:
#Get venues for all neighborhoods in our dataset
toronto_venues = getNearbyVenues(names=df_toronto['Neighborhood'], latitudes = df_toronto['Latitude'], longitudes = df_toronto['Longitude'])

The Beaches
The Danforth West , Riverdale
India Bazaar , The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park , Summerhill East
Summerhill West , Rathnelly , South Hill , Forest Hill SE , Deer Park
Rosedale
St. James Town , Cabbagetown
Church and Wellesley
Regent Park , Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond , Adelaide , King
Harbourfront East , Union Station , Toronto Islands
Toronto Dominion Centre , Design Exchange
Commerce Court , Victoria Hotel
Roselawn
Forest Hill North & West
The Annex , North Midtown , Yorkville
University of Toronto , Harbord
Kensington Market , Chinatown , Grange Park
CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport
Stn A PO Boxes
First Canadian Place , Underground city
Christie
Dufferin , Dovercourt Village
Little Portugal , Trinity
Brockton , Parkdale Village , Exhibition Place
High Park , 

In [34]:
#Check size of resulting dataframe
toronto_venues.shape

(1624, 7)

In [35]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


In [36]:
#Number of venues per neighborhood
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton , Parkdale Village , Exhibition Place",23,23,23,23,23,23
Business reply mail Processing CentrE,17,17,17,17,17,17
"CN Tower , King and Spadina , Railway Lands , Harbourfront West , Bathurst Quay , South Niagara , Island airport",17,17,17,17,17,17
Central Bay Street,65,65,65,65,65,65
Christie,18,18,18,18,18,18
Church and Wellesley,72,72,72,72,72,72
"Commerce Court , Victoria Hotel",100,100,100,100,100,100
Davisville,37,37,37,37,37,37


In [37]:
#Number of unique venue categories
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 232 uniques categories.


In [38]:
#print out the list of categories
toronto_venues['Venue Category'].unique()[:100]

array(['Trail', 'Health Food Store', 'Pub', 'Neighborhood', 'Coffee Shop',
       'Cosmetics Shop', 'Greek Restaurant', 'Italian Restaurant',
       'Ice Cream Shop', 'Yoga Studio', 'Brewery',
       'Fruit & Vegetable Store', 'Dessert Shop', 'Restaurant',
       'Pizza Place', 'Juice Bar', 'Bookstore', 'Diner',
       'Bubble Tea Shop', 'Furniture / Home Store', 'Grocery Store',
       'Spa', 'Bakery', 'Caribbean Restaurant', 'Café',
       'Indian Restaurant', 'Japanese Restaurant', 'Lounge',
       'Frozen Yogurt Shop', 'American Restaurant', 'Liquor Store', 'Gym',
       'Fish & Chips Shop', 'Fast Food Restaurant', 'Sushi Restaurant',
       'Park', 'Pet Store', 'Steakhouse', 'Burrito Place',
       'Movie Theater', 'Sandwich Place', 'Board Shop',
       'Food & Drink Shop', 'Fish Market', 'Gay Bar',
       'Seafood Restaurant', 'Cheese Shop', 'Middle Eastern Restaurant',
       'Comfort Food Restaurant', 'Stationery Store', 'Wine Bar',
       'Thai Restaurant', 'Coworking Space', 

In [39]:
# check if the results contain "Bar"
"Bar" in toronto_venues['Venue Category'].unique()

True

### Analyze Each Neighborhood

In [40]:
# one hot encoding
to_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
to_onehot['Neighborhoods'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [to_onehot.columns[-1]] + list(to_onehot.columns[:-1])
to_onehot = to_onehot[fixed_columns]

print(to_onehot.shape)
to_onehot.head()

(1624, 233)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
to_grouped = to_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(to_grouped.shape)
to_grouped

(39, 233)


,Neighborhoods,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Women's Store,Yoga Studio
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.043478
2,Business reply mail Processing CentrE,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.058824
3,"CN Tower , King and Spadina , Railway Lands , ...",0.058824,0.058824,0.058824,0.117647,0.176471,0.117647,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.015385,0.000000,0.000000,0.000000,0.000000,0.015385
5,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013889,0.000000,0.00,...,0.013889,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.027778
7,"Commerce Court , Victoria Hotel",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.000000
8,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.027027,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [42]:
len(to_grouped[to_grouped["Bar"] > 0])

19

Create a new dataframe to find Bars only

In [43]:
to_bar = to_grouped[["Neighborhoods","Bar"]]

In [44]:
to_bar.head()

,Neighborhoods,Bar
0,Berczy Park,0.000000
1,"Brockton , Parkdale Village , Exhibition Place",0.043478
2,Business reply mail Processing CentrE,0.000000
3,"CN Tower , King and Spadina , Railway Lands , ...",0.058824
4,Central Bay Street,0.015385


# Cluster Neighborhoods

Run k-means to cluster the neighborhoods in Toronto into 3 clusters.

In [45]:
# set number of clusters
toclusters = 3

to_clustering = to_bar.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=toclusters, random_state=0).fit(to_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 1, 2, 0, 0, 0, 2, 0])

In [46]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
to_merged = to_bar.copy()

# add clustering labels
to_merged["Cluster Labels"] = kmeans.labels_

In [47]:
to_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
to_merged.head()

,Neighborhood,Bar,Cluster Labels
0,Berczy Park,0.000000,0
1,"Brockton , Parkdale Village , Exhibition Place",0.043478,2
2,Business reply mail Processing CentrE,0.000000,0
3,"CN Tower , King and Spadina , Railway Lands , ...",0.058824,1
4,Central Bay Street,0.015385,2


In [48]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
to_merged = to_merged.join(toronto_venues.set_index("Neighborhood"), on="Neighborhood")

print(to_merged.shape)
to_merged.head()

(1624, 9)


,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.0,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
0,Berczy Park,0.0,0,43.644771,-79.373306,The Keg Steakhouse + Bar - Esplanade,43.646712,-79.374768,Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Fresh On Front,43.647815,-79.374453,Vegetarian / Vegan Restaurant
0,Berczy Park,0.0,0,43.644771,-79.373306,Meridian Hall,43.646292,-79.376022,Concert Hall
0,Berczy Park,0.0,0,43.644771,-79.373306,Hockey Hall Of Fame (Hockey Hall of Fame),43.646974,-79.377323,Museum


In [49]:
# sort the results by Cluster Labels
print(to_merged.shape)
to_merged.sort_values(["Cluster Labels"], inplace=True)
to_merged

(1624, 9)


,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.000000,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,Stonemill Bread,43.648668,-79.371610,Bakery
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,The Works Gourmet Burger Bistro,43.648742,-79.374142,Restaurant
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,Buster's Sea Cove,43.648495,-79.371462,Seafood Restaurant
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,Market Street Catch,43.648501,-79.371808,Seafood Restaurant
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,C'est What,43.648426,-79.373439,Beer Bar
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,St. Lawrence Market Plaza,43.649169,-79.372330,Art Gallery
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,Loaded Pierogi,43.647965,-79.373427,Comfort Food Restaurant
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,The Omni King Edward Hotel,43.649191,-79.376006,Hotel
31,Stn A PO Boxes,0.000000,0,43.646435,-79.374846,Mos Mos Coffee,43.648159,-79.378745,Café


### vizualize the clusters

In [50]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [51]:
# create map
map_clusters = folium.Map(location=[lat_toronto, lon_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(toclusters)
ys = [i+x+(i*x)**2 for i in range(toclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(to_merged['Neighborhood Latitude'], to_merged['Neighborhood Longitude'], to_merged['Neighborhood'], to_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster))
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [52]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

# Examine Clusters

### Cluster 0 Analysis

In [53]:
#Cluster 0
Cluster0 = to_merged.loc[to_merged['Cluster Labels'] == 0]
Cluster0

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Berczy Park,0.00,0,43.644771,-79.373306,LCBO,43.642944,-79.372440,Liquor Store
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,Stonemill Bread,43.648668,-79.371610,Bakery
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,The Works Gourmet Burger Bistro,43.648742,-79.374142,Restaurant
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,Buster's Sea Cove,43.648495,-79.371462,Seafood Restaurant
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,Market Street Catch,43.648501,-79.371808,Seafood Restaurant
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,C'est What,43.648426,-79.373439,Beer Bar
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,St. Lawrence Market Plaza,43.649169,-79.372330,Art Gallery
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,Loaded Pierogi,43.647965,-79.373427,Comfort Food Restaurant
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,The Omni King Edward Hotel,43.649191,-79.376006,Hotel
31,Stn A PO Boxes,0.00,0,43.646435,-79.374846,Mos Mos Coffee,43.648159,-79.378745,Café


In [54]:
Cluster0['Bar'].describe()

count    977.000000
mean       0.004094
std        0.004998
min        0.000000
25%        0.000000
50%        0.000000
75%        0.010000
max        0.010753
Name: Bar, dtype: float64

In [55]:
Cluster0['Bar'].sum()

4.0

### Cluster 1 Analysis

In [56]:
#Cluster 1
Cluster1 = to_merged.loc[to_merged['Cluster Labels'] == 1]
Cluster1

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,River Tai Restaurant,43.662902,-79.403167,Chinese Restaurant
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Red Fish Blue Fish Creative Cafe,43.662827,-79.402817,Café
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,The Clubhouse,43.658246,-79.399894,Sandwich Place
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Gyubee,43.667088,-79.400571,Japanese Restaurant
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Hart House Theatre,43.663571,-79.394616,Theater
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Yasu,43.662837,-79.403217,Japanese Restaurant
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Innis Cafe,43.665401,-79.399715,Café
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Harbord Bakery & Calandria,43.662519,-79.404443,Bakery
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Athletic Centre,43.662487,-79.400657,College Gym
38,"University of Toronto , Harbord",0.057143,1,43.662696,-79.400049,Harvest Kitchen,43.662714,-79.404330,Restaurant


In [57]:
Cluster1['Bar'].describe()

count    122.000000
mean       0.073770
std        0.017917
min        0.057143
25%        0.057143
50%        0.062500
75%        0.097561
max        0.097561
Name: Bar, dtype: float64

In [58]:
Cluster1['Bar'].sum()

8.999999999999998

### Cluster 2 Analysis

In [59]:
#Cluster 2
Cluster2 = to_merged.loc[to_merged['Cluster Labels'] == 2]
Cluster2

,Neighborhood,Bar,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Toronto Athletic Club,43.647360,-79.381177,Gym
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Design Exchange,43.647972,-79.380104,Art Gallery
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Bymark,43.647217,-79.381252,Restaurant
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,DAVIDsTEA,43.650547,-79.383385,Tea Room
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Petit Four Bakery,43.647744,-79.379588,Sandwich Place
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Bulldog On The Block,43.650652,-79.384141,Coffee Shop
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Assembly Chef's Hall,43.650579,-79.383412,Food Court
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,deKEFIR,43.650433,-79.379393,Ice Cream Shop
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Bosk at Shangri-La,43.649023,-79.385826,Asian Restaurant
11,"First Canadian Place , Underground city",0.020000,2,43.648429,-79.382280,Four Seasons Centre for the Performing Arts,43.650592,-79.385806,Concert Hall


In [60]:
Cluster2['Bar'].describe()

count    525.000000
mean       0.024762
std        0.007929
min        0.015385
25%        0.020000
50%        0.020000
75%        0.027027
max        0.045455
Name: Bar, dtype: float64

In [61]:
Cluster2['Bar'].sum()

13.0

# Conclusion

By the analysis of the above 3 clusters, it is noted that most of the Bars are in Cluster 2 which are around First Canadian Place , Underground city, Harbourfront East , Union Station , and Toronto Islands. The lowest number of bars are in Cluster 0 areas which are Toronto Dominion Centre , Design Exchange, Commerce Court , Victoria Hotel, Garden District, Ryerson, etc. Hence, there are good opportunities to open the bar in these areas of Cluster 0 as competition seems to be low and it could attract the nearby crowd more often than compared to other clusters. Based on the results obtained by the analysis, it can be concluded that the neighborhoods of Cluster 0 might be a good location as there are not a lot of Bars in these areas for people to choose from. Therefore, this project recommends the entrepreneur to open a Bar in a suitable neighborhood of Cluster 0. 